# Author Modeling Pipeline

This notebook implements a 5-stage pipeline for modeling an author's characteristic writing patterns through few-shot example curation.

## Methodology

Rather than extracting explicit style rules, this approach models the author's **decision-making patterns** and **sensibility**, then curates exemplary passages for tacit transmission via few-shot learning.

## Pipeline Stages

1. **Stage 1: Analytical Mining** - Analyze sample texts through three lenses:
   - Implied Author: Sensibility and stance
   - Decision Patterns: Compositional choices
   - Functional Texture: How surface features serve purpose

2. **Stage 2: Cross-Text Synthesis** - Synthesize each dimension across multiple samples to identify stable patterns

3. **Stage 3: Field Guide Construction** - Integrate all three syntheses into unified recognition criteria and density rubric

4. **Stage 4: Passage Evaluation** - Evaluate passages from corpus using field guide (1-5 density rating)

5. **Stage 5: Example Set Construction** - Curate 3-4 high-density passages that balance density, diversity, and complementarity

## Note

This pipeline is author-agnostic. The Russell corpus is used as a test case, but the same approach applies to any author.

## 1. Setup & Dependencies

In [4]:
!pip install -r requirements.txt

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import os
from pathlib import Path
from pprint import pprint
from typing import List, Dict, Any

from belletrist import (
    LLM,
    LLMConfig,
    PromptMaker,
    DataSampler,
    ResultStore,
    # Stage 1 configs
    ImpliedAuthorConfig,
    DecisionPatternConfig,
    FunctionalTextureConfig,
    # Stage 2 configs
    ImpliedAuthorSynthesisConfig,
    DecisionPatternSynthesisConfig,
    TexturalSynthesisConfig,
    # Stage 3 config
    AuthorModelDefinitionConfig,
    FieldGuideConstructionConfig,
    # Stage 4 config
    PassageEvaluationConfig,
    # Stage 5 config
    ExampleSetConstructionConfig,
    # Utilities
    extract_paragraph_windows,
    extract_logical_sections,
    get_full_sample_as_passage,
    parse_passage_evaluation,
    parse_example_set_selection,
    validate_passage_evaluation,
    validate_example_set_selection,
)

print("Dependencies imported successfully")

Dependencies imported successfully


### Initialize Base Objects

In [6]:
# Initialize LLM
llm_config = LLMConfig(
    #model="mistral/mistral-large-2411",  # Change as needed
    model='together_ai/moonshotai/Kimi-K2-Instruct-0905',
    api_key=os.environ.get('TOGETHER_AI_API_KEY'),
    temperature=0.7
)
llm = LLM(llm_config)

# Initialize prompt maker
prompt_maker = PromptMaker()

# Initialize data sampler (Russell corpus)
data_dir = Path("data/russell")
sampler = DataSampler(data_dir)

# Initialize result store
store = ResultStore("russell_author_modeling_kimi.db")

In [7]:
#store.reset('all')

### Configuration: Select Samples for Analysis

For Stages 1-3, we'll analyze 3-5 samples to extract stable cross-text patterns.

In [8]:
# Select samples for Stage 1-3 analysis
# These should be diverse, substantial passages (500-800 words recommended)
ANALYSIS_SAMPLES = [
    "sample_001",
    "sample_002",
    "sample_003",
    "sample_004",
    "sample_005",
    # Add more as needed for richer cross-text synthesis
]

# For this demo, we'll generate samples if they don't exist
# In production, you might use specific pre-selected samples
NUM_SAMPLES = 5
SAMPLE_PARAGRAPH_LENGTH = 10

print(f"Will analyze {NUM_SAMPLES} samples through Stages 1-3")

Will analyze 5 samples through Stages 1-3


## 2. Stage 1: Analytical Mining

Run three specialized analyses on each sample:
- **Implied Author**: What sensibility emerges from the prose?
- **Decision Patterns**: What compositional choices are made at key junctures?
- **Functional Texture**: How do surface features serve purpose?

### Generate or Load Samples

In [9]:
# Generate samples if they don't exist
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    
    # Skip if already saved
    if store.get_sample(sample_id):
        print(f"✓ {sample_id} already gathered")
        continue
        
    # Generate new sample
    segment = sampler.sample_segment(p_length=SAMPLE_PARAGRAPH_LENGTH)
    store.save_segment(sample_id, segment)
    print(f"Generated {sample_id}")

# Display first sample for reference
sample = store.get_sample("sample_001")
print(f"\nSample 001 preview:\n{sample['text']}...")

Generated sample_001
Generated sample_002
Generated sample_003
Generated sample_004
Generated sample_005

Sample 001 preview:
All names of places--London, England, Europe, the Earth, the Solar
System--similarly involve, when used, descriptions which start from some
one or more particulars with which we are acquainted. I suspect that
even the Universe, as considered by metaphysics, involves such a
connexion with particulars. In logic, on the contrary, where we are
concerned not merely with what does exist, but with whatever might or
could exist or be, no reference to actual particulars is involved.

It would seem that, when we make a statement about something only known
by description, we often _intend_ to make our statement, not in the form
involving the description, but about the actual thing described. That
is to say, when we say anything about Bismarck, we should like, if we
could, to make the judgement which Bismarck alone can make, namely,
the judgement of which he himself is a co

### Run Stage 1A: Implied Author Analysis

In [10]:
# Analyze each sample for implied author
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analyst_name = ImpliedAuthorConfig.analyst_name()
    
    # Check if analysis already exists (resume support)
    if store.get_analysis(sample_id, analyst_name):
        print(f"{sample_id}: {analyst_name} analysis already complete")
        continue
    
    # Get sample text
    sample = store.get_sample(sample_id)
    
    # Generate prompt
    config = ImpliedAuthorConfig(text=sample['text'])
    prompt = prompt_maker.render(config)
    
    # Run LLM
    print(f"{sample_id}: Running {analyst_name} analysis...")
    response = llm.complete(prompt)
    
    # Save analysis
    store.save_analysis(
        sample_id=sample_id,
        analyst=analyst_name,
        output=response.content,
        model=response.model
    )
    print(f"{sample_id}: {analyst_name} analysis complete")

print("\nStage 1A complete: All samples analyzed for implied author")

sample_001: Running implied_author analysis...
sample_001: implied_author analysis complete
sample_002: Running implied_author analysis...
sample_002: implied_author analysis complete
sample_003: Running implied_author analysis...
sample_003: implied_author analysis complete
sample_004: Running implied_author analysis...
sample_004: implied_author analysis complete
sample_005: Running implied_author analysis...
sample_005: implied_author analysis complete

Stage 1A complete: All samples analyzed for implied author


### Run Stage 1B: Decision Pattern Analysis

In [11]:
# Analyze each sample for decision patterns
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analyst_name = DecisionPatternConfig.analyst_name()
    
    if store.get_analysis(sample_id, analyst_name):
        print(f"{sample_id}: {analyst_name} analysis already complete")
        continue
    
    sample = store.get_sample(sample_id)
    config = DecisionPatternConfig(text=sample['text'])
    prompt = prompt_maker.render(config)
    
    print(f"{sample_id}: Running {analyst_name} analysis...")
    response = llm.complete(prompt)
    
    store.save_analysis(
        sample_id=sample_id,
        analyst=analyst_name,
        output=response.content,
        model=response.model
    )
    print(f"{sample_id}: {analyst_name} analysis complete")

print("\nStage 1B complete: All samples analyzed for decision patterns")

sample_001: Running decision_pattern analysis...
sample_001: decision_pattern analysis complete
sample_002: Running decision_pattern analysis...
sample_002: decision_pattern analysis complete
sample_003: Running decision_pattern analysis...
sample_003: decision_pattern analysis complete
sample_004: Running decision_pattern analysis...
sample_004: decision_pattern analysis complete
sample_005: Running decision_pattern analysis...
sample_005: decision_pattern analysis complete

Stage 1B complete: All samples analyzed for decision patterns


### Run Stage 1C: Functional Texture Analysis

In [12]:
# Analyze each sample for functional texture
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analyst_name = FunctionalTextureConfig.analyst_name()
    
    if store.get_analysis(sample_id, analyst_name):
        print(f"{sample_id}: {analyst_name} analysis already complete")
        continue
    
    sample = store.get_sample(sample_id)
    config = FunctionalTextureConfig(text=sample['text'])
    prompt = prompt_maker.render(config)
    
    print(f"{sample_id}: Running {analyst_name} analysis...")
    response = llm.complete(prompt)
    
    store.save_analysis(
        sample_id=sample_id,
        analyst=analyst_name,
        output=response.content,
        model=response.model
    )
    print(f"{sample_id}: {analyst_name} analysis complete")

print("\nStage 1C complete: All samples analyzed for functional texture")

sample_001: Running functional_texture analysis...
sample_001: functional_texture analysis complete
sample_002: Running functional_texture analysis...
sample_002: functional_texture analysis complete
sample_003: Running functional_texture analysis...
sample_003: functional_texture analysis complete
sample_004: Running functional_texture analysis...
sample_004: functional_texture analysis complete
sample_005: Running functional_texture analysis...
sample_005: functional_texture analysis complete

Stage 1C complete: All samples analyzed for functional texture


### Inspect Stage 1 Results

In [13]:
# Display completion status
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    sample, analyses = store.get_sample_with_analyses(sample_id)
    print(f"{sample_id}: {len(analyses)} analyses complete")
    for analyst_name in analyses.keys():
        print(f"  - {analyst_name}")

# Optionally display one analysis
print("\n--- Sample Implied Author Analysis (first 500 chars) ---")
sample, analyses = store.get_sample_with_analyses("sample_005")
implied_author = analyses.get(ImpliedAuthorConfig.analyst_name(), "Not found")
print(f"Total chars: {len(implied_author)}")
print(implied_author)

sample_001: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_002: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_003: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_004: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author
sample_005: 3 analyses complete
  - decision_pattern
  - functional_texture
  - implied_author

--- Sample Implied Author Analysis (first 500 chars) ---
Total chars: 6688
PART 1: DIMENSIONAL ANALYSIS

1. RELATIONSHIP TO MATERIAL  
Observation: The writer treats the subject as something still in motion, holding it at arm’s length to watch it change shape.  They neither celebrate nor mourn; they test possibilities, keep score of counter-possibilities, and allow the picture to stay porous.

Evidence  
a) “There are signs that the paralysis is merely temporary.”  
   – The clause “there are signs” keeps the verdict c

## 3. Stage 2: Cross-Text Synthesis

Synthesize each analytical dimension across samples to identify stable patterns.

### Stage 2A: Implied Author Synthesis

In [14]:
# Collect all implied author analyses
implied_author_analyses = {}
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analysis = store.get_analysis(sample_id, ImpliedAuthorConfig.analyst_name())
    if analysis:
        implied_author_analyses[sample_id] = analysis

# Check if synthesis already exists
synthesis_type = ImpliedAuthorSynthesisConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Implied author synthesis already exists: {existing[0]}")
    implied_author_synthesis_id = existing[0]
else:
    # Generate synthesis prompt
    config = ImpliedAuthorSynthesisConfig(
        implied_author_analyses=implied_author_analyses
    )
    prompt = prompt_maker.render(config)
    
    # Run synthesis
    print("Running implied author synthesis...")
    response = llm.complete(prompt)
    
    # Save synthesis
    sample_contributions = [
        (sample_id, ImpliedAuthorConfig.analyst_name())
        for sample_id in implied_author_analyses.keys()
    ]
    
    implied_author_synthesis_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=sample_contributions,
        config=config
    )
    print(f"Synthesis saved: {implied_author_synthesis_id}")

# Display preview
synth = store.get_synthesis(implied_author_synthesis_id)
print(f"\nPreview (first 400 chars):\n{synth['output'][:400]}...")

Running implied author synthesis...
Synthesis saved: implied_author_synthesis_001

Preview (first 400 chars):
### PART 1: THE STABLE CORE  

**Fundamental Stance Toward Ideas and Inquiry**  
Across every text, the author treats knowledge as something that must be *shown being assembled*, not delivered pre-packaged. Whether the topic is sense-data, relativity, belief, education, or Russian art, the writer begins with the reader’s ordinary certainties, then slowly loosens the screws. The method is consisten...


### Stage 2B: Decision Pattern Synthesis

In [15]:
# Collect all decision pattern analyses
decision_pattern_analyses = {}
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analysis = store.get_analysis(sample_id, DecisionPatternConfig.analyst_name())
    if analysis:
        decision_pattern_analyses[sample_id] = analysis

# Check if synthesis already exists
synthesis_type = DecisionPatternSynthesisConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Decision pattern synthesis already exists: {existing[0]}")
    decision_pattern_synthesis_id = existing[0]
else:
    config = DecisionPatternSynthesisConfig(
        decision_pattern_analyses=decision_pattern_analyses
    )
    prompt = prompt_maker.render(config)
    
    print("Running decision pattern synthesis...")
    response = llm.complete(prompt)
    
    sample_contributions = [
        (sample_id, DecisionPatternConfig.analyst_name())
        for sample_id in decision_pattern_analyses.keys()
    ]
    
    decision_pattern_synthesis_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=sample_contributions,
        config=config
    )
    print(f"Synthesis saved: {decision_pattern_synthesis_id}")

synth = store.get_synthesis(decision_pattern_synthesis_id)
print(f"\nPreview (first 400 chars):\n{synth['output'][:400]}...")

Running decision pattern synthesis...
Synthesis saved: decision_pattern_synthesis_001

Preview (first 400 chars):
### PART 1: COMPOSITIONAL PROBLEM TAXONOMY

1. **Opening Complex Abstract Claims** – How to begin discussions of dense philosophical or theoretical concepts without losing the reader immediately  
2. **Bridging Concrete ↔ Abstract** – Moving between everyday experience and abstract principle without creating a jarring leap  
3. **Staging Reader Resistance** – Anticipating and incorporating the rea...


### Stage 2C: Textural Synthesis

In [16]:
# Collect all functional texture analyses
textural_analyses = {}
for i in range(NUM_SAMPLES):
    sample_id = f"sample_{i+1:03d}"
    analysis = store.get_analysis(sample_id, FunctionalTextureConfig.analyst_name())
    if analysis:
        textural_analyses[sample_id] = analysis

# Check if synthesis already exists
synthesis_type = TexturalSynthesisConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Textural synthesis already exists: {existing[0]}")
    textural_synthesis_id = existing[0]
else:
    config = TexturalSynthesisConfig(
        textural_analyses=textural_analyses
    )
    prompt = prompt_maker.render(config)
    
    print("Running textural synthesis...")
    response = llm.complete(prompt)
    
    sample_contributions = [
        (sample_id, FunctionalTextureConfig.analyst_name())
        for sample_id in textural_analyses.keys()
    ]
    
    textural_synthesis_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=sample_contributions,
        config=config
    )
    print(f"Synthesis saved: {textural_synthesis_id}")

synth = store.get_synthesis(textural_synthesis_id)
print(f"\nPreview (first 400 chars):\n{synth['output'][:400]}...")

Running textural synthesis...
Synthesis saved: textural_synthesis_001

Preview (first 400 chars):
# PART 1: SENTENCE ARCHITECTURE

This author's sentences feel like a hand reaching forward while the mind keeps catching the sleeve to add one more reservation. The characteristic shape is **back-loaded complexity**: a short, steady main clause arrives early, then a tail of qualifications, parenthetical hedges, and illustrative riders accumulates, each appendix testing whether the first statement ...


In [17]:
implied_synth = store.get_synthesis(implied_author_synthesis_id)
decision_synth = store.get_synthesis(decision_pattern_synthesis_id)
textural_synth = store.get_synthesis(textural_synthesis_id)

# Check if author model definition already exists
synthesis_type = AuthorModelDefinitionConfig.synthesis_type()
existing = store.list_syntheses(synthesis_type)

if existing:
    print(f"Author model definition already exists: {existing[0]}")
    author_model_id = existing[0]
else:
    # Generate author model definition
    config = AuthorModelDefinitionConfig(
        implied_author_synthesis=implied_synth['output'],
        decision_pattern_synthesis=decision_synth['output'],
        textural_synthesis=textural_synth['output']
    )
    prompt = prompt_maker.render(config)

    print("Constructing author model definition...")
    response = llm.complete(prompt)

    # Save with parent linkage (no direct sample contributions)
    author_model_id = store.save_synthesis(
        synthesis_type=synthesis_type,
        output=response.content,
        model=response.model,
        sample_contributions=[],  # Inherits from parents
        config=config,
        parent_synthesis_id=implied_author_synthesis_id  # Link to one parent
    )
    print(f"Author model definition saved: {author_model_id}")

# Display preview
author_model = store.get_synthesis(author_model_id)
print(f"\nTotal chars of Author Model: {len(author_model['output'])}")
print(f"\nAuthor Model Preview (first 800 chars):\n{author_model['output'][:800]}...")

# Export to filesystem for use in generation
output_dir = Path("outputs/author_modeling")
output_dir.mkdir(parents=True, exist_ok=True)

author_model_path = output_dir / f"{author_model_id}.txt"
store.export_synthesis(
    synthesis_id=author_model_id,
    output_path=author_model_path,
    metadata_format='yaml'
)
print(f"\nAuthor model exported to: {author_model_path}")

Constructing author model definition...
Author model definition saved: author_model_definition_001

Total chars of Author Model: 8799

Author Model Preview (first 800 chars):
# AUTHOR MIND MODEL: BERTRAND RUSSELL  
*(for LLM inhabitation)*

---

## PART 1: THE GENERATIVE STANCE  
*(≈480 words, second-person)*

You sit down to write convinced that the reader already possesses every tool needed to follow you—only the *order* of the tools is wrong. Your job is not to add new apparatus but to unscrew the hinges and lay them on the table so the mechanism can be watched in motion. You therefore begin with what “we all” grant: the sun will rise, the train is late, the child builds a sand-pie. These are not concessions to simplicity; they are the immovable fulcrums on which you will lever the whole world of abstraction. You *trust* the concrete moment more than the grand theory, because a theory can hide its cracks while a tiger in the street forces the reader’s body t...

Author model exporte